Retrieve experiment results

In [56]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [8, 9, 10, 13]
basedirs = [join(get_output_dir(), 'nips', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        source = config['source']
        weights = config['dataset_weights']
        seed = config['seed']
        model = config['model']
        std = config['with_std']
        dropout = config['input_dropout_rate']
        rank = info['rank']
        score = info['score']
        res = {'alpha': alpha, 'seed': seed, 'rank': rank, 'datasets': datasets, 'source': source, 'dropout': dropout,
              'model': model, 'std': std}
        for key, value in score.items():
            res[key] = value
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [57]:
res

,alpha,datasets,dropout,model,rank,seed,source,std,test_archi,test_brainomics,test_camcan,test_hcp,test_la5c,test_mean,train_archi,train_brainomics,train_camcan,train_hcp,train_la5c,train_mean
0,0.00000,archi,0.25,factored,30,3705352044,hcp_rs_concat,True,0.864103,NaN,NaN,NaN,NaN,0.864103,1.000000,NaN,NaN,NaN,NaN,1.000000
1,0.00000,archi,0.25,factored,30,3581115008,hcp_rs_concat,True,0.860684,NaN,NaN,NaN,NaN,0.860684,1.000000,NaN,NaN,NaN,NaN,1.000000
2,0.00000,archi,0.25,factored,30,3765953013,hcp_rs_concat,True,0.839316,NaN,NaN,NaN,NaN,0.839316,1.000000,NaN,NaN,NaN,NaN,1.000000
3,0.00000,archi,0.25,factored,30,415692431,hcp_rs_concat,True,0.838462,NaN,NaN,NaN,NaN,0.838462,0.998889,NaN,NaN,NaN,NaN,0.998889
4,0.00000,archi,0.25,factored,30,2770472721,hcp_rs_concat,True,0.848718,NaN,NaN,NaN,NaN,0.848718,1.000000,NaN,NaN,NaN,NaN,1.000000
5,0.00000,archi,0.25,factored,30,2603172742,hcp_rs_concat,True,0.858120,NaN,NaN,NaN,NaN,0.858120,1.000000,NaN,NaN,NaN,NaN,1.000000
6,0.00000,archi,0.25,factored,30,3436456541,hcp_rs_concat,True,0.856410,NaN,NaN,NaN,NaN,0.856410,1.000000,NaN,NaN,NaN,NaN,1.000000
7,0.00000,archi__hcp,0.25,logistic,53,3765953013,hcp_rs_concat,True,0.811966,NaN,NaN,0.913043,NaN,0.873452,1.000000,NaN,NaN,0.956685,NaN,0.958958
8,0.00000,archi__hcp,0.25,logistic,53,2770472721,hcp_rs_concat,True,0.816239,NaN,NaN,0.925702,NaN,0.882826,1.000000,NaN,NaN,0.954162,NaN,0.956567
9,0.00000,archi__hcp,0.25,logistic,53,415692431,hcp_rs_concat,True,0.805983,NaN,NaN,0.907887,NaN,0.867918,1.000000,NaN,NaN,0.956696,NaN,0.958967


In [58]:
df_agg = res.groupby(by=['datasets', 'model', 'std', 'dropout', 'source', 'alpha']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)

In [59]:
results = {}
for dataset in ['archi', 'brainomics', 'camcan', 'la5c']:
    results[dataset] = df_agg.loc[[dataset, '%s__hcp' % dataset]]['test_%s' % dataset].groupby(level=('datasets', 'model', 'std', 'dropout', 'source')).aggregate('max')

In [60]:
results['archi']

mean       std
datasets   model    std   dropout source                                    
archi      factored True  0.25    hcp_rs                  0.850427  0.009902
                                  hcp_rs_concat           0.848376  0.011058
                                  hcp_rs_positive         0.855214  0.007486
                                  hcp_rs_positive_single  0.857009  0.010081
           logistic False 0.00    hcp_rs_positive         0.849060  0.033827
                                  hcp_rs_positive_single  0.851709  0.036202
                          0.25    hcp_rs_positive         0.842735  0.009636
                                  hcp_rs_positive_single  0.845726  0.012401
                    True  0.00    hcp_rs                  0.817778  0.012582
                                  hcp_rs_concat           0.815299  0.016133
                          0.25    hcp_rs                  0.809060  0.008866
                                  hcp_rs_concat           0.807778  0.009280
archi__hcp factored True  0.25    hcp_rs_concat           0.863628  0.011145
           logistic True  0.25    hcp_rs                  0.818205  0.010615
                                  hcp_rs_concat           0.812906  0.009288
                                  hcp_rs_positive         0.807179  0.007615
                                  hcp_rs_positive_single  0.810513  0.010476

In [61]:
results['brainomics']

mean  \
datasets        model    std   dropout source                             
brainomics      factored True  0.25    hcp_rs                  0.873641   
                                       hcp_rs_concat           0.876714   
                                       hcp_rs_positive         0.888889   
                                       hcp_rs_positive_single  0.885697   
                logistic False 0.00    hcp_rs_positive         0.877896   
                                       hcp_rs_positive_single  0.884043   
                               0.25    hcp_rs_positive         0.855083   
                                       hcp_rs_positive_single  0.859929   
                         True  0.00    hcp_rs                  0.826359   
                                       hcp_rs_concat           0.835343   
                               0.25    hcp_rs                  0.827541   
                                       hcp_rs_concat           0.833452   
brainomics__hcp factored True  0.25    hcp_rs_concat           0.889598   
                logistic True  0.25    hcp_rs                  0.851418   
                                       hcp_rs_concat           0.852482   
                                       hcp_rs_positive         0.843026   
                                       hcp_rs_positive_single  0.845154   

                                                                    std  
datasets        model    std   dropout source                            
brainomics      factored True  0.25    hcp_rs                  0.008045  
                                       hcp_rs_concat           0.012973  
                                       hcp_rs_positive         0.010747  
                                       hcp_rs_positive_single  0.011187  
                logistic False 0.00    hcp_rs_positive         0.019582  
                                       hcp_rs_positive_single  0.030693  
                               0.25    hcp_rs_positive         0.019416  
                                       hcp_rs_positive_single  0.016784  
                         True  0.00    hcp_rs                  0.026269  
                                       hcp_rs_concat           0.025933  
                               0.25    hcp_rs                  0.017492  
                                       hcp_rs_concat           0.017421  
brainomics__hcp factored True  0.25    hcp_rs_concat           0.013188  
                logistic True  0.25    hcp_rs                  0.013210  
                                       hcp_rs_concat           0.014539  
                                       hcp_rs_positive         0.014248  
                                       hcp_rs_positive_single  0.012157

In [55]:
results['camcan']

mean       std
datasets    model    std   dropout source                                    
camcan      factored True  0.25    hcp_rs                  0.617756  0.009496
                                   hcp_rs_concat           0.614983  0.008465
                                   hcp_rs_positive         0.616568  0.007954
                                   hcp_rs_positive_single  0.618548  0.009613
            logistic False 0.00    hcp_rs_positive         0.620594  0.014535
                                   hcp_rs_positive_single  0.618944  0.017145
                           0.25    hcp_rs_positive         0.619472  0.010807
                                   hcp_rs_positive_single  0.616964  0.009845
                     True  0.00    hcp_rs                  0.622838  0.011560
                                   hcp_rs_concat           0.622904  0.010418
                           0.25    hcp_rs                  0.618944  0.008126
                                   hcp_rs_concat           0.617954  0.007684
camcan__hcp factored True  0.25    hcp_rs_concat           0.620858  0.009819
            logistic True  0.25    hcp_rs                  0.619010  0.008302
                                   hcp_rs_concat           0.620858  0.007110
                                   hcp_rs_positive         0.618746  0.008151
                                   hcp_rs_positive_single  0.621188  0.009026

In [62]:
results['la5c']

mean       std
datasets  model    std   dropout source                                    
la5c      factored True  0.25    hcp_rs                  0.591615  0.013856
                                 hcp_rs_concat           0.589566  0.011674
                                 hcp_rs_positive         0.592723  0.011739
                                 hcp_rs_positive_single  0.594835  0.010814
          logistic False 0.00    hcp_rs_positive         0.597580  0.012393
                                 hcp_rs_positive_single  0.601779  0.015372
                         0.25    hcp_rs_positive         0.597923  0.011290
                                 hcp_rs_positive_single  0.600587  0.011529
                   True  0.00    hcp_rs                  0.567779  0.013485
                                 hcp_rs_concat           0.567269  0.013819
                         0.25    hcp_rs                  0.572666  0.013055
                                 hcp_rs_concat           0.569131  0.012134
la5c__hcp logistic True  0.25    hcp_rs                  0.573540  0.012448
                                 hcp_rs_concat           0.569320  0.012006
                                 hcp_rs_positive         0.580037  0.011770
                                 hcp_rs_positive_single  0.579448  0.013453

In [16]:
indices = df_agg.groupby(level='model').aggregate('idxmax')[('test_mean', 'mean')].values
df_agg = df_agg.loc[indices]

In [19]:
df_agg

archi_weight      brainomics_weight      hcp_weight       \
                          mean  std              mean  std       mean  std   
model    alpha                                                               
factored 0.000720            1  0.0                 1  0.0          1  0.0   
logistic 0.000037            1  0.0                 1  0.0          1  0.0   
trace    0.000720            1  0.0                 1  0.0          1  0.0   

                    rank             seed                  ...    test_mean  \
                    mean  std        mean           std    ...         mean   
model    alpha                                             ...                
factored 0.000720  32.75  0.5  1765347879  1.157716e+09    ...     0.908035   
logistic 0.000037  68.00  0.0  1765347879  1.157716e+09    ...     0.904840   
trace    0.000720  46.75  1.5  1765347879  1.157716e+09    ...     0.906209   

                            train_archi           train_brainomics       \
                        std        mean       std             mean  std   
model    alpha                                                            
factored 0.000720  0.005450    0.994231  0.003909              1.0  0.0   
logistic 0.000037  0.003965    0.996581  0.002315              1.0  0.0   
trace    0.000720  0.002955    0.995726  0.003042              1.0  0.0   

                  train_hcp           train_mean            
                       mean       std       mean       std  
model    alpha                                              
factored 0.000720  0.967621  0.000834   0.970825  0.000591  
logistic 0.000037  0.973666  0.000737   0.976353  0.000681  
trace    0.000720  0.968437  0.000645   0.971646  0.000635  

[3 rows x 26 columns]

In [17]:
df_agg[['test_archi', 'test_hcp', 'test_brainomics', 'rank', 'test_mean']]

test_archi            test_hcp           test_brainomics  \
                        mean       std      mean       std            mean   
model    alpha                                                               
factored 0.000720   0.866453  0.018834  0.929692  0.007968        0.919031   
logistic 0.000037   0.863034  0.015167  0.931756  0.009415        0.904846   
trace    0.000720   0.875214  0.014672  0.928729  0.008556        0.900709   

                              rank      test_mean            
                        std   mean  std      mean       std  
model    alpha                                               
factored 0.000720  0.017809  32.75  0.5  0.908035  0.005450  
logistic 0.000037  0.013802  68.00  0.0  0.904840  0.003965  
trace    0.000720  0.014119  46.75  1.5  0.906209  0.002955

Compare results with 3 datasets

In [18]:
transfer_df

test_archi  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0            0.036752   
                                                                 0.5            0.862393   
                                                                 1.0            0.864103   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000003 0    0.0            0.024786   
                                                                 0.5            0.863248   
                                                                 1.0            0.864957   
                                                                 1.5            0.876068   
                                                                 2.0            0.852991   
                                                   0.000010 0    0.0            0.026496   
                                                                 0.5            0.864957   
                                                                 1.0            0.868376   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000032 0    0.0            0.023932   
                                                                 0.5            0.869231   
                                                                 1.0            0.872650   
                                                                 1.5            0.876068   
                                                                 2.0            0.859829   
                                                   0.000100 0    0.0            0.044444   
                                                                 0.5            0.879487   
                                                                 1.0            0.873504   
                                                                 1.5            0.876923   
                                                                 2.0            0.864103   
                                                   0.000316 0    0.0            0.043590   
                                                                 0.5            0.875214   
                                                                 1.0            0.870085   
                                                                 1.5            0.878632   
                                                                 2.0            0.864957   
                                                   0.001000 0    0.0            0.029060   
                                                                 0.5            0.860684   
                                                                 1.0            0.865812   
                                                                 1.5            0.876923   
                                                                 2.0            0.863248   
                                                   0.003162 0    0.0            0.013675   
                                                                 0.5            0.786325   
                                                                 1.0            0.827350   
                                                                 1.5            0.840171   
                                                                 2.0            0.653846   
                                                   0.010000 0    0.0            0.042735   
                              

In [13]:
this_df

hcp_weight  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000003 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000010 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000032 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000100 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000316 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.001000 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.003162 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.010000 0    0.0                 2.0   
                              

In [5]:
indices.values

array([(False, 'hcp_rs_positive_single', 'logistic', 'archi', 0.0, 0.0001),
       (False, 'hcp_rs_positive_single', 'trace', 'archi', 0.001, 0.0),
       (False, 'hcp_rs_positive_single', 'trace', 'archi__hcp', 0.00031622776601683794, 0.0)], dtype=object)

In [6]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df[('test_mean', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

In [7]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [8]:
result_df

rank  \
                                                                                                         mean   
target               normalized source                 model    datasets               alpha    beta            
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  17.05   
                                                       trace    brainomics             0.003162 0.0000   9.50   
                                                                brainomics__hcp        0.003162 0.0000  19.05   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  29.05   
                                                       trace    archi                  0.001000 0.0000  18.85   
                                                                archi__hcp             0.000316 0.0000  38.05   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  51.45   

                                                                                                                  \
                                                                                                             std   
target               normalized source                 model    datasets               alpha    beta               
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  0.223607   
                                                       trace    brainomics             0.003162 0.0000  0.512989   
                                                                brainomics__hcp        0.003162 0.0000  0.825578   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  0.223607   
                                                       trace    archi                  0.001000 0.0000  0.670820   
                                                                archi__hcp             0.000316 0.0000  1.050063   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  0.944513   

                                                                                                              seed  \
                                                                                                              mean   
target               normalized source                 model    datasets               alpha    beta                 
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  2330081002   
                                                       trace    brainomics             0.003162 0.0000  2330081002   
                                                                brainomics__hcp        0.003162 0.0000  2330081002   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  2330081002   
                                                       trace    archi                  0.001000 0.0000  2330081002   
                                                                archi__hcp             0.000316 0.0000  2330081002   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  2330081002   

                                                                                                                      \
                                                                                                                 std   
target               normalized source                 model    datasets               alpha    beta                   
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  1.242420e+09   
                                                       trace    brainomics             0.003162 0.0000  1.242420e+09   
                                

In [21]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                   mean   
normalized source                 model datasets               alpha    beta              
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   71.000000   
                                                               0.000001 0.0   71.000000   
                                                               0.000003 0.0   71.000000   
                                                               0.000010 0.0   71.000000   
                                                               0.000032 0.0   71.000000   
                                                               0.000100 0.0   70.666667   
                                                               0.000316 0.0   52.000000   
                                                               0.001000 0.0   28.000000   
                                                               0.003162 0.0   18.333333   
                                                               0.010000 0.0   10.000000   
                                                               0.031623 0.0    3.666667   
                                                               0.100000 0.0    1.000000   
                                                               0.316228 0.0    0.000000   
                                                               1.000000 0.0    0.000000   

                                                                                        \
                                                                                   std   
normalized source                 model datasets               alpha    beta             
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   0.000000   
                                                               0.000001 0.0   0.000000   
                                                               0.000003 0.0   0.000000   
                                                               0.000010 0.0   0.000000   
                                                               0.000032 0.0   0.000000   
                                                               0.000100 0.0   0.577350   
                                                               0.000316 0.0   1.000000   
                                                               0.001000 0.0   0.000000   
                                                               0.003162 0.0   1.154701   
                                                               0.010000 0.0   0.000000   
                                                               0.031623 0.0   0.577350   
                                                               0.100000 0.0   0.000000   
                                                               0.316228 0.0   0.000000   
                                                               1.000000 0.0   0.000000   

                                                                                    seed  \
                                                                                    mean   
normalized source                 model datasets               alpha    beta               
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   1243615929   
                                                               0.000001 0.0   1243615929   
                                                               0.000003 0.0   1243615929   
                                                               0.000010 0.0   1243615929   
                                                               0.000032 0.0   1243615929   
                                                               0.000100 0.0   1243615929   
                                                               0.000316 0.0   1243615929   
                                                               0.001000 0.0   124361

In [22]:
df

NameError: name 'df' is not defined